# TensorBoard - Logování obrázků 🏞 a textů 📜
Do TensorBoardu můžeme logovat i texty a obrázky. To se může hodit například na ukázku vstupů a výstupů, získaných metadat v průběhu trénování či na diagnostiku problémů.

In [ ]:
import tensorflow as tf
from datetime import datetime
import numpy as np
import io
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
%load_ext tensorboard

# removes logs from previous runs
!rm -rf ./logs/ 

## 📜 Texty
K logování textů slouží **TensorFlow Text Summary API**. Postup je následující:
* 📁 definujeme, kde se budou ukládat logy
* ✍️ vytvoříme file writer pomocí `tf.summary.create_file_writer()`
* ☑️ text zalogujeme pomocí file writeru a funkce `tf.summary.text()`

In [ ]:
# sets up a timestamped log directory
def create_log_dir(subfolder):
    return 'logs/' + subfolder + '/' + datetime.now().strftime('%Y%m%d-%H%M%S')


log_dir = create_log_dir('text')
# creates a file writer for the log directory
file_writer = tf.summary.create_file_writer(log_dir)

In [ ]:
text = 'Visualizations are great! 😍'

with file_writer.as_default():
    # logs the text
    tf.summary.text('my first text', text, step=0)

Zalogovaný text se pak ukáže v dashboardu **Text**.

In [ ]:
%tensorboard --logdir logs/text

Pokud je během trénování potřeba logovat text z více míst, lze logy organizovat do jmenných prostorů. Pozor na to, že pokud budete logovat hodně, TensorBoard zobrazí jen tolik dat, kolik uzná za vhodné. Toto chování lze ovlivnit použitím flagu `--samples_per_plugin`, kde můžeme přesně specifikovat, kolik logů se má zobrazit.

TensorBoard také podporuje vykreslování Markdownu 😍.

In [ ]:
log_dir = create_log_dir('text')
file_writer = tf.summary.create_file_writer(log_dir)

markdown_texts = [
    '### This is a headline',
    '**Bold** and *italic* text',
    """
| *hello* | *there* |
|---------|---------|
| this    | is      |
| a       | table   |
    """
]

with file_writer.as_default():
    # creates a scope (name space) for text organisation
    with tf.name_scope('first scope'):
        for step in range(3):
            tf.summary.text("markdown", markdown_texts[step], step=step)
            tf.summary.text("random number", 'My random number is {}!'.format(np.random.randint(100)), step=step)
    with tf.name_scope("second scope"):
        tf.summary.text("step 3 announcement", "Step 3 is the best 👌!", step=3)    

Zkusme nyní aktualizovat předcházející TensorBoard okno, a potom spustit následující buňku. Liší se právě v použití `--samples_per_plugin` tagu.

In [ ]:
%tensorboard --logdir logs/text --samples_per_plugin 'text=1'

## 🏞 Obrázky
Logování obrázků si ukážeme na stejném příkladu, jaký jsme použili v předchozím notebooku.

🧠 Neuronová síť, která predikuje nad datasetem MNIST ✍️.

In [ ]:
# this code is copied from previous notebook
mnist = tf.keras.datasets.mnist

(X_train, y_train),(X_test, y_test) = mnist.load_data()

X_train = X_train / 255.0
X_test = X_test / 255.0

Když se podíváme na vstupní data, vidíme, že jeden obrázek je reprezentován 2D polem (přesněji rank-2 tensor; šířka, výška).

In [ ]:
print("Image shape: ", X_train[0].shape)

TensorBoard očekává obrázek jako 4D pole (rank-4 tensor; batch size, výška, šířka, počet barevných kanálů). Proto musíme obrázek nejprve transformovat ♻️ a teprve potom zalogovat. Jelikož chceme logovat jen jeden obrázek, **batch size bude 1**. V předchozím notebooku jsme si říkali, že obrázky jsou černobílé, tedy mají jen **jeden barevný kanál**.

In [ ]:
# reshapes the image for the Image Summary API
img = np.reshape(X_train[0], (-1, 28, 28, 1))
print('first pixel value: {}'.format(img[0][0][0][0]))
print('some pixel value: {}'.format(img[0][10][11][0]))
print('last pixel value: {}'.format(img[0][27][27][0]))

K logování obrázků slouží **TensorFlow Image Summary API**. Postup je následující:
* 📁 definujeme, kde se budou ukládat logy
* ✍️ vytvoříme file writer pomocí `tf.summary.create_file_writer()`
* ☑️ obrázek ve vhodném formátu zalogujeme pomocí file writeru a funkce `tf.summary.image()`

In [ ]:
log_dir = create_log_dir('image')
file_writer = tf.summary.create_file_writer(log_dir)

with file_writer.as_default():
    tf.summary.image("First training image", img, step=0)

In [ ]:
%tensorboard --logdir logs/image

Obrázek se zobrazí v dashboardu Images a má upravenou velikost. Pokud si chcete prohlédnout obrázky ve skutečné velikosti, je třeba označit checkbox *Show actual image size* v levém panelu.

Zalogovat více než jeden obrázek lze velmi podobně:

In [ ]:
imgs = np.reshape(X_train[1:11], (-1, 28, 28, 1))

with file_writer.as_default():
    tf.summary.image("Other training images", imgs, max_outputs=10, step=0)

### 🧰 Praktická ukázka - logování matice záměn po každé epoše

In [ ]:
# these functions are copied from previous notebook

def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

def compile_model(model):
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

Pokud chceme zalogovat obrázek, který není tensor, tedy například matplotlib graf, potřebujeme jej nejprve zkonvertovat.

In [ ]:
# converts matplotlib figure to png and then to tensor flow image
def convert_to_tf_image(figure):
    # saves the plot to memmory as PNG
    buffer = io.BytesIO()
    figure.savefig(buffer, format='png')
    buffer.seek(0)
    
    # closing the figure so it is not displayed in cell output
    plt.close(figure)
    
    # converts PNG buffer to TF image
    image = tf.image.decode_png(buffer.getvalue(), channels=4)
    
    # adds the batch dimension
    image = tf.expand_dims(image, 0)
    return image

Nyní vytvoříme funkci, která se bude volat na konci každé epochy. Ve funkci vypočítáme predikce pro validační množinu (X_test), vytvoříme matici záměn, tu zkonvertujeme do správného formátu a poté zalogujeme do TensourBoardu.

Pak nám ještě zbývá vytvořit callback 🤙, který tuto funkci zavolá na konci každé epochy, a předat jej modelu ve funkci fit.

In [ ]:
log_dir = create_log_dir('image')
file_writer = tf.summary.create_file_writer(log_dir)

def log_confusion_matrix(epoch, logs):
    # creates predictions
    y_pred_probabilities = model.predict(X_test)
    y_pred = np.argmax(y_pred_probabilities, axis=1)
    
    # creates confusion matrix in TensorBoard compatible format
    cm = ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
    cm_tensor = convert_to_tf_image(cm.figure_)
    
    # logs the image
    with file_writer.as_default():
        tf.summary.image("Confusion Matrix", cm_tensor, step=epoch)

# per-epoch callback
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
model = create_model()
compile_model(model)

model.fit(
    x=X_train, 
    y=y_train, 
    epochs=10, 
    verbose=0,
    validation_data=(X_test, y_test),
    callbacks=[cm_callback]
)